<a href="https://colab.research.google.com/github/etodasha/PRA-M2.875-2020.2/blob/main/PRA_DL_Xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 69 bytes


In [ ]:
!kaggle datasets download "jordidelatorreuoc/kaggle-plant-pathology-2021-modificat"

 98% 955M/970M [00:05<00:00, 129MB/s]
100% 970M/970M [00:05<00:00, 170MB/s]


In [ ]:
!unzip kaggle-plant-pathology-2021-modificat.zip

In [ ]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import keras 
import os
import matplotlib.pylab as plt
import cv2
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout, ZeroPadding2D, GlobalAveragePooling2D,BatchNormalization
from keras.optimizers import Adam, SGD,RMSprop,Adadelta,Adagrad,Adamax,Nadam,Ftrl
from keras.layers.experimental.preprocessing import Resizing
from keras.callbacks import *
from tensorflow.keras.layers import Input,Dense,Conv2D,Add
from tensorflow.keras.layers import SeparableConv2D,ReLU
from tensorflow.keras.layers import BatchNormalization,MaxPool2D
from tensorflow.keras.layers import GlobalAvgPool2D
from tensorflow.keras import Model

DATA AUGMENTATION + TRAIN VALIDATION


In [ ]:
datagen_train = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8,1.2],
    zoom_range=[0.5,1.2],
    rotation_range=20,
    rescale=1./255,
    validation_split=0.2) 

train_generator = datagen_train.flow_from_directory( 
    'train',
    subset='training')

validation_generator = datagen_train.flow_from_directory(
    'train',
    subset='validation'
    ,shuffle=False)



Found 7800 images belonging to 6 classes.
Found 1950 images belonging to 6 classes.


In [ ]:
from keras import backend as K

def custom_recall(y_true, y_pred):
  TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  recall = TP / (Positives+K.epsilon())
  return recall 

def custom_precision(y_true, y_pred):
  TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = TP / (Pred_Positives+K.epsilon())
  return precision

def custom_f1(y_true, y_pred):    
    precision, recall = custom_precision(y_true, y_pred), custom_recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def conv_bn(x, filters, kernel_size, strides=1):
    
    x = Conv2D(filters=filters, 
               kernel_size = kernel_size, 
               strides=strides, 
               padding = 'same', 
               use_bias = False)(x)
    x = BatchNormalization()(x)
    return x

def sep_bn(x, filters, kernel_size, strides=1):
    
    x = SeparableConv2D(filters=filters, 
                        kernel_size = kernel_size, 
                        strides=strides, 
                        padding = 'same', 
                        use_bias = False)(x)
    x = BatchNormalization()(x)
    return x

def entry_flow(x):
    
    x = Resizing(299,299)(x)

    x = conv_bn(x, filters =32, kernel_size =3, strides=2)
    x = ReLU()(x)
    x = conv_bn(x, filters =64, kernel_size =3, strides=1)
    tensor = ReLU()(x)
    
    x = sep_bn(tensor, filters = 128, kernel_size =3)
    x = ReLU()(x)
    x = sep_bn(x, filters = 128, kernel_size =3)
    x = MaxPool2D(pool_size=3, strides=2, padding = 'same')(x)
    
    tensor = conv_bn(tensor, filters=128, kernel_size = 1,strides=2)
    x = Add()([tensor,x])
    
    x = ReLU()(x)
    x = sep_bn(x, filters =256, kernel_size=3)
    x = ReLU()(x)
    x = sep_bn(x, filters =256, kernel_size=3)
    x = MaxPool2D(pool_size=3, strides=2, padding = 'same')(x)
    
    tensor = conv_bn(tensor, filters=256, kernel_size = 1,strides=2)
    x = Add()([tensor,x])
    
    x = ReLU()(x)
    x = sep_bn(x, filters =728, kernel_size=3)
    x = ReLU()(x)
    x = sep_bn(x, filters =728, kernel_size=3)
    x = MaxPool2D(pool_size=3, strides=2, padding = 'same')(x)
    
    tensor = conv_bn(tensor, filters=728, kernel_size = 1,strides=2)
    x = Add()([tensor,x])
    return x

def middle_flow(tensor):
    
    for _ in range(8):
        x = ReLU()(tensor)
        x = sep_bn(x, filters = 728, kernel_size = 3)
        x = ReLU()(x)
        x = sep_bn(x, filters = 728, kernel_size = 3)
        x = ReLU()(x)
        x = sep_bn(x, filters = 728, kernel_size = 3)
        x = ReLU()(x)
        tensor = Add()([tensor,x])
        return tensor

def exit_flow(tensor):
    
    x = ReLU()(tensor)
    x = sep_bn(x, filters = 728,  kernel_size=3)
    x = ReLU()(x)
    x = sep_bn(x, filters = 1024,  kernel_size=3)
    x = MaxPool2D(pool_size = 3, strides = 2, padding ='same')(x)
    
    tensor = conv_bn(tensor, filters =1024, kernel_size=1, strides =2)
    x = Add()([tensor,x])
    
    x = sep_bn(x, filters = 1536,  kernel_size=3)
    x = ReLU()(x)
    x = sep_bn(x, filters = 2048,  kernel_size=3)
    x = GlobalAvgPool2D()(x)
    
    x = Dense (units = 1000, activation = 'relu')(x)

    x = Dense (units = 6, activation = 'softmax')(x)
    
    return x


input = Input(shape = (332, 498,3))
x = entry_flow(input)
x = middle_flow(x)
output = exit_flow(x)

model = Model (inputs=input, outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 332, 498, 3) 0                                            
__________________________________________________________________________________________________
resizing (Resizing)             (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 150, 150, 32) 864         resizing[0][0]                   
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 150, 150, 32) 128         conv2d[0][0]                     
______________________________________________________________________________________________

In [ ]:
# Xception
model.compile(optimizer = Adam(learning_rate=0.0001), 
              loss = "categorical_crossentropy", 
              metrics = ['accuracy',custom_f1,custom_recall,custom_precision])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_custom_f1', mode='max', patience=15, 
                                               restore_best_weights=True, verbose=2)

filepath = "/content/gdrive/My Drive/Xception_epochs:{epoch:03d}-val_f1:{val_custom_f1:.3f}.hdf5"

checkpoint_callback = ModelCheckpoint(filepath= filepath, monitor='val_custom_f1', mode='max',
                                                         save_weights_only=True, save_best_only=True)


model_history = model.fit(train_generator,
                              validation_data = validation_generator,
                              epochs=100,
                              batch_size=512,
                              callbacks=[early_stopping, checkpoint_callback]) 

Epoch 1/100
244/244 [==============================] - 268s 947ms/step - loss: 1.2461 - accuracy: 0.5463 - custom_f1: 0.4634 - custom_recall: 0.3599 - custom_precision: 0.6817 - val_loss: 1.7091 - val_accuracy: 0.4103 - val_custom_f1: 0.0000e+00 - val_custom_recall: 0.0000e+00 - val_custom_precision: 0.0000e+00
Epoch 2/100
244/244 [==============================] - 230s 940ms/step - loss: 0.9122 - accuracy: 0.6704 - custom_f1: 0.6512 - custom_recall: 0.5725 - custom_precision: 0.7607 - val_loss: 2.0588 - val_accuracy: 0.4103 - val_custom_f1: 0.4067 - val_custom_recall: 0.4037 - val_custom_precision: 0.4098
Epoch 3/100
244/244 [==============================] - 230s 940ms/step - loss: 0.7817 - accuracy: 0.7190 - custom_f1: 0.7068 - custom_recall: 0.6457 - custom_precision: 0.7843 - val_loss: 1.4265 - val_accuracy: 0.5267 - val_custom_f1: 0.4906 - val_custom_recall: 0.4641 - val_custom_precision: 0.5344
Epoch 4/100
244/244 [==============================] - 229s 939ms/step - loss: 0.6943

In [ ]:
#xception 0.8811 ('/content/gdrive/My Drive/Xcpetion_epochs:22-val_f1:0.881.hdf5')